# Consignment Data and CFT Analysis

In [135]:
## importing library and packages

import numpy as np 
import pandas as pd
import math

In [136]:
pd.set_option('display.max_row', 500)
pd.set_option('display.max_column', 60)

In [176]:
data = pd.read_csv("C:/Users/saiba/OneDrive/Documents/Data Analysis Projects/Project 11 - Identify Consignments With Wrong Weight -Volume During Pick-up/Data/P2_consignment_data.csv")
data.head()

,id,created_date,cnote,client_id,weight,Volume,total_boxes,industry_type,QC_Validation,delivered,Delivery_Date,cpm
0,32123,15-01-2019 00:00,8000522895,1112,13.0,3.23,1,APPAREL,Yes,1,19-01-2019 17:01,1
1,32124,15-01-2019 00:00,8000119761,1113,122.0,23.30,14,3PL,Yes,1,17-01-2019 11:18,1
2,32125,15-01-2019 00:01,8000522921,1112,19.0,3.23,1,APPAREL,Yes,1,18-01-2019 10:38,1
3,32126,15-01-2019 00:01,8000574330,1114,68.0,11.16,4,3PL,Yes,1,18-01-2019 17:42,0
4,32127,15-01-2019 00:02,1002478259,1115,1075.0,73.35,50,CHEMICALS,Yes,1,22-01-2019 17:29,1


### basic info

In [157]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86663 entries, 0 to 86662
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             86663 non-null  int64  
 1   created_date   86663 non-null  object 
 2   cnote          86663 non-null  object 
 3   client_id      86663 non-null  int64  
 4   weight         86663 non-null  float64
 5   Volume         86663 non-null  float64
 6   total_boxes    86663 non-null  int64  
 7   industry_type  86663 non-null  object 
 8   QC_Validation  86663 non-null  object 
 9   delivered      86663 non-null  int64  
 10  Delivery_Date  62536 non-null  object 
 11  cpm            86663 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 7.9+ MB


In [158]:
data.client_id.value_counts()

1151    2682
1655    2503
1159    1782
1176    1590
1160    1496
        ... 
2305       1
1781       1
2144       1
2300       1
2407       1
Name: client_id, Length: 1296, dtype: int64

In [266]:
np.min(data.client_id)

1112

In [159]:
## changing datetime dtype

data['created_date'] = pd.to_datetime(data['created_date'])

In [160]:
data['created_date'].dtype

dtype('<M8[ns]')

In [161]:
data['Delivery_Date'] = pd.to_datetime(data['Delivery_Date'])  #changing delivery_date time to datetime object

In [162]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86663 entries, 0 to 86662
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             86663 non-null  int64         
 1   created_date   86663 non-null  datetime64[ns]
 2   cnote          86663 non-null  object        
 3   client_id      86663 non-null  int64         
 4   weight         86663 non-null  float64       
 5   Volume         86663 non-null  float64       
 6   total_boxes    86663 non-null  int64         
 7   industry_type  86663 non-null  object        
 8   QC_Validation  86663 non-null  object        
 9   delivered      86663 non-null  int64         
 10  Delivery_Date  62536 non-null  datetime64[ns]
 11  cpm            86663 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(5), object(3)
memory usage: 7.9+ MB


## step - 1 :
**Calculating the CFT ( = Weight / Volume ) of consignments at the row level to perform row-level aggregations**

In [177]:
## calculating CFT

data['CFT'] = data['weight']/data['Volume']

In [178]:
np.min(data['CFT']), np.max(data['CFT']), data['CFT'].mode()

(0.04800076801228819,
 inf,
 0    4.188482
 Name: CFT, dtype: float64)

In [181]:
data.drop(data[data['CFT'] == float('inf')].index, axis=0, inplace=True)

In [182]:
data.query('CFT == inf')

,id,created_date,cnote,client_id,weight,Volume,total_boxes,industry_type,QC_Validation,delivered,Delivery_Date,cpm,CFT


In [185]:
data['CFT'].describe()

count    86659.000000
mean        12.529473
std         55.684550
min          0.048001
25%          4.266786
50%          7.030907
75%         13.031172
max       9800.000000
Name: CFT, dtype: float64

## step - 2
**Summarizing the consignment data at the client and industry level**

In [186]:
## client summary

client_summary = data.groupby('client_id').agg({
    'id' : 'count',
    'weight' : np.sum,
    'Volume' : np.sum,
    'CFT' : np.mean}).reset_index().rename(columns = {"CFT" : "avg_CFT"})

In [187]:
## overview of client_summary
client_summary.head()

,client_id,id,weight,Volume,avg_CFT
0,1112,1186,36762.22,6555.87,5.951837
1,1113,51,3804.60,588.76,8.136285
2,1114,503,32251.00,5070.25,7.352325
3,1115,91,52530.80,3727.19,55.339088
4,1116,781,129519.87,13675.42,10.337240


In [188]:
## industry summary

industry_summary = data.groupby('industry_type').agg({
    'id' : 'count',
    'weight' : np.sum, 
    'Volume' : np.sum,
    'CFT' : np.mean}).reset_index().rename(columns = {"CFT" : "avg_CFT"})

In [189]:
## industry_summary

industry_summary.head()

,industry_type,id,weight,Volume,avg_CFT
0,3PL,2417,249954.488,35976.67,10.066926
1,APPAREL,14025,1611937.687,367519.24,9.901644
2,AUTOMOTIVE,2247,526586.868,45712.67,15.323694
3,CHEMICALS,689,371927.295,26946.11,19.954314
4,ECOMMERCE,4775,687595.056,152588.08,4.812405


In [190]:
industry_summary.columns

Index(['industry_type', 'id', 'weight', 'Volume', 'avg_CFT'], dtype='object')

In [314]:
## combine client and industry summary

client_industry_summary = data.groupby(['industry_type', 'client_id']).agg({
    'id' : 'count',
    'weight' : np.sum, 
    'Volume' : np.sum,
    'CFT' : np.mean}).reset_index().rename(columns = {"CFT" : "avg_CFT"})

In [315]:
client_industry_summary

,industry_type,client_id,id,weight,Volume,avg_CFT
0,3PL,1113,51,3804.6,588.76,8.136285
1,3PL,1114,503,32251.0,5070.25,7.352325
2,3PL,1124,65,12972.7,2964.62,8.141684
3,3PL,1137,16,2555.0,334.84,7.374463
4,3PL,1141,92,3486.0,1151.40,3.314798
...,...,...,...,...,...,...
1291,RETAIL,2344,1,60.0,10.00,6.000000
1292,RETAIL,2359,1,1680.0,260.81,6.441471
1293,RETAIL,2366,2,83.0,5.53,15.317606
1294,RETAIL,2401,1,125.0,9.54,13.102725


## step - 3
**Calculate statistical summary for consignment CFT ( Mean, Max Percentile)**

In [193]:
## summary statistics

client_industry_CFT_stats = client_industry_summary['avg_CFT'].describe().reset_index().rename(columns = {'index': 'parameter', 'avg_CFT' : 'values'})

In [194]:
client_industry_CFT_stats

,parameter,values
0,count,1296.000000
1,mean,15.094871
2,std,39.324153
3,min,0.458175
4,25%,6.111337
5,50%,9.353199
6,75%,15.885971
7,max,1175.123838


## step - 4 
**Calculate the Industry level upper and lower limits of CFT**

In [311]:
client_industry_summary

,Industry Type,Client ID,id,Weight,Volume,Average CFT
0,3PL,1113,51,3804.6,588.76,8.136285
1,3PL,1114,503,32251.0,5070.25,7.352325
2,3PL,1124,65,12972.7,2964.62,8.141684
3,3PL,1137,16,2555.0,334.84,7.374463
4,3PL,1141,92,3486.0,1151.40,3.314798
...,...,...,...,...,...,...
1291,RETAIL,2344,1,60.0,10.00,6.000000
1292,RETAIL,2359,1,1680.0,260.81,6.441471
1293,RETAIL,2366,2,83.0,5.53,15.317606
1294,RETAIL,2401,1,125.0,9.54,13.102725


In [316]:
## query an observation :

## What is the average CFT of a chemical client - client ID - 11 ?

data.query('industry_type == "CHEMICALS" & client_id == 1115')['CFT'].mean()

55.33908825250881

In [333]:
## new dataframe containing Industry and CFT values only

CFT_analysis = client_industry_summary[['industry_type', 'avg_CFT']]

In [334]:
## overview
CFT_analysis.head()

,industry_type,avg_CFT
0,3PL,8.136285
1,3PL,7.352325
2,3PL,8.141684
3,3PL,7.374463
4,3PL,3.314798


In [335]:
## summerise

CFT_analysis = CFT_analysis.groupby('industry_type')['avg_CFT'].describe().reset_index()

In [336]:
## upper and lower limits of CFT values on each industry

# tile_75_CFT_industry = CFT_analysis.groupby('industry_type')['CFT'].quantile(0.75).reset_index()
# tile_25_CFT_industry = CFT_analysis.groupby('industry_type')['CFT'].quantile(0.25).reset_index()

In [337]:
## adding columns to new dataframe

# CFT_analysis = pd.merge(tile_25_CFT_industry, tile_75_CFT_industry, on = 'Industry', how = 'left')

# CFT_analysis['percentile_75'] = tile_75_CFT_industry
# CFT_analysis['percentile_25'] = tile_25_CFT_industry

In [338]:
CFT_analysis

,industry_type,count,mean,std,min,25%,50%,75%,max
0,3PL,43.0,9.647218,6.502072,2.523951,5.677022,7.765152,10.833887,30.964523
1,APPAREL,124.0,7.669866,6.504756,1.856764,4.366021,5.932954,8.329453,41.169170
2,AUTOMOTIVE,31.0,26.687157,33.100472,0.458175,8.165505,14.799739,32.548234,168.526263
3,CHEMICALS,20.0,15.203284,10.795015,6.406183,7.860512,13.353849,18.279041,55.339088
4,ECOMMERCE,34.0,8.118426,4.927077,2.188911,4.577852,6.947784,10.722583,25.568593
5,ELECTRONICS,111.0,11.163653,8.502844,2.726103,6.150313,8.387494,12.477393,44.290150
6,FMCG,45.0,15.874048,33.926970,1.161500,5.151920,9.489177,14.741249,232.485056
7,INDUSTRIAL,440.0,18.733458,62.391687,1.376273,6.852958,10.844966,17.823290,1175.123838
8,MEDIA & PUBLICATIONS,45.0,17.077632,20.899901,3.087991,9.704788,14.090069,17.700813,148.281070
9,NON-ECOMMERCE,250.0,16.161824,20.478789,1.311266,6.422057,9.809772,17.841632,188.034188


In [339]:
## IQR limits

# 25% percentile
Q1 = CFT_analysis['25%']
# 75% percentile
Q3 = CFT_analysis['75%']

## inter quartile range(difference between 75 and 25 percentile)
IQR = Q3 - Q1  
CFT_analysis['IQR'] = IQR
## upper limit
CFT_analysis['upper_limit'] = Q3 + 1.5 * IQR

## lower limit
CFT_analysis['lower_limit'] = Q1 - 1.5 * IQR

In [340]:
CFT_analysis

,industry_type,count,mean,std,min,25%,50%,75%,max,IQR,upper_limit,lower_limit
0,3PL,43.0,9.647218,6.502072,2.523951,5.677022,7.765152,10.833887,30.964523,5.156865,18.569185,-2.058276
1,APPAREL,124.0,7.669866,6.504756,1.856764,4.366021,5.932954,8.329453,41.169170,3.963432,14.274600,-1.579126
2,AUTOMOTIVE,31.0,26.687157,33.100472,0.458175,8.165505,14.799739,32.548234,168.526263,24.382728,69.122326,-28.408587
3,CHEMICALS,20.0,15.203284,10.795015,6.406183,7.860512,13.353849,18.279041,55.339088,10.418529,33.906834,-7.767282
4,ECOMMERCE,34.0,8.118426,4.927077,2.188911,4.577852,6.947784,10.722583,25.568593,6.144731,19.939679,-4.639243
5,ELECTRONICS,111.0,11.163653,8.502844,2.726103,6.150313,8.387494,12.477393,44.290150,6.327080,21.968013,-3.340306
6,FMCG,45.0,15.874048,33.926970,1.161500,5.151920,9.489177,14.741249,232.485056,9.589330,29.125244,-9.232075
7,INDUSTRIAL,440.0,18.733458,62.391687,1.376273,6.852958,10.844966,17.823290,1175.123838,10.970333,34.278790,-9.602541
8,MEDIA & PUBLICATIONS,45.0,17.077632,20.899901,3.087991,9.704788,14.090069,17.700813,148.281070,7.996024,29.694849,-2.289248
9,NON-ECOMMERCE,250.0,16.161824,20.478789,1.311266,6.422057,9.809772,17.841632,188.034188,11.419575,34.970994,-10.707305


In [202]:
## adding 85percentile result to the dataframe

CFT_analysis['85%'] = client_industry_summary['avg_CFT'].quantile(0.85)

## step - 5
**Compare the Consignment CFT with IQR limits and Mark the outliers in a separate sheet**

In [236]:
# creating new dataframe for raw Consignment data only
data_CFT_ = data['CFT'].describe().reset_index()

In [237]:
data_CFT = data_CFT_.transpose()

In [238]:
data_CFT

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
CFT,86659.0,12.529473,55.68455,0.048001,4.266786,7.030907,13.031172,9800.0


In [ ]:
data_CFT.drop(index = 'index', inplace = True)

In [241]:
data_CFT = data_CFT.rename(columns = {0: 'count', 1:'mean', 2:'stddev', 3:'min', 4:'tile_25', 5:'tile_50', 6:'tile_75', 7:'max'})

In [243]:
data_CFT

,count,mean,stddev,min,tile_25,tile_50,tile_75,max
CFT,86659.0,12.529473,55.68455,0.048001,4.266786,7.030907,13.031172,9800.0


In [246]:
## analysis of raw data CFT

data_CFT['IQR'] = data_CFT['tile_75'] - data_CFT.tile_25   ## IQR column

data_CFT['upper_limit'] = data_CFT['tile_75'] + 1.5 * data_CFT.IQR   ## upper limit
data_CFT['lower_limit'] = data_CFT['tile_25'] - 1.5 * data_CFT.IQR   ## lower limit

In [247]:
data_CFT

,count,mean,stddev,min,tile_25,tile_50,tile_75,max,IQR,upper_limit,lower_limit
CFT,86659.0,12.529473,55.68455,0.048001,4.266786,7.030907,13.031172,9800.0,8.764386,26.177751,-8.879792


## Expected Output

### output - 1

In [269]:
client_industry_summary

,industry_type,client_id,id,weight,Volume,avg_CFT
0,3PL,1113,51,3804.6,588.76,8.136285
1,3PL,1114,503,32251.0,5070.25,7.352325
2,3PL,1124,65,12972.7,2964.62,8.141684
3,3PL,1137,16,2555.0,334.84,7.374463
4,3PL,1141,92,3486.0,1151.40,3.314798
...,...,...,...,...,...,...
1291,RETAIL,2344,1,60.0,10.00,6.000000
1292,RETAIL,2359,1,1680.0,260.81,6.441471
1293,RETAIL,2366,2,83.0,5.53,15.317606
1294,RETAIL,2401,1,125.0,9.54,13.102725


In [298]:
## output-1
output_1 = client_industry_summary.rename(columns = {
                                 'industry_type' : "Industry Type",
                                 'client_id' : "Client ID",
                                 'is' : "#CN Count",
                                 'weight' : "Weight",
                                 'Volume' : "Volume",
                                 'avg_CFT' : "Average CFT"})

In [300]:
## setting `Client ID` as index column
output_1 = output_1.set_index("Client ID")

In [301]:
output_1

,Industry Type,id,Weight,Volume,Average CFT
Client ID,,,,,
1113,3PL,51,3804.6,588.76,8.136285
1114,3PL,503,32251.0,5070.25,7.352325
1124,3PL,65,12972.7,2964.62,8.141684
1137,3PL,16,2555.0,334.84,7.374463
1141,3PL,92,3486.0,1151.40,3.314798
...,...,...,...,...,...
2344,RETAIL,1,60.0,10.00,6.000000
2359,RETAIL,1,1680.0,260.81,6.441471
2366,RETAIL,2,83.0,5.53,15.317606


In [302]:
# saving the outputs
file_location = "C:/Users/saiba/OneDrive/Documents/Data Analysis Projects/Project 11 - Identify Consignments With Wrong Weight -Volume During Pick-up/Data"
output_1.to_csv(file_location + "/output_1.csv")

### CFT analysis

In [341]:
## modifying the dataframe
CFT_analysis = CFT_analysis.drop(['min','std', 'IQR', 'upper_limit', 'lower_limit'], axis = 1)

In [342]:
## renaming the columns and setting the index columns
CFT_analysis = CFT_analysis.rename({
    'industry_type' : 'Industry_type',
    'count' : "#CN count",
    'mean' : "Mean",
    'max' : "Max",
    '25%' : "Percentile_25",
    '50%' : "Percentile_50",
    '75%' : "Percentile_75",
    '85%' : "Percentile_85"}).set_index('industry_type').reset_index()

In [343]:
CFT_analysis.columns

Index(['industry_type', 'count', 'mean', '25%', '50%', '75%', 'max'], dtype='object')

In [344]:
# saving the outputs
file_location = "C:/Users/saiba/OneDrive/Documents/Data Analysis Projects/Project 11 - Identify Consignments With Wrong Weight -Volume During Pick-up/Data"
CFT_analysis.to_csv(file_location + "/CFT_analysis.csv")